In [14]:
import pandas as pd
import numpy as np

## Technical indicators for price data

In [15]:
df = pd.read_csv("../../Data/Tezos/Processed/MarketAndPrice.csv")
df.head()

,date,current_price,market_cap,total_volume,close,high,low,open
0,2018-07-03,2.937866,0.000000e+00,3.172538e+06,2.361,3.589,2.033,3.083
1,2018-07-04,2.047985,0.000000e+00,1.426545e+07,2.255,2.401,2.025,2.345
2,2018-07-05,1.969392,1.196384e+09,6.492796e+06,1.940,2.396,1.880,2.257
3,2018-07-06,1.528033,9.282631e+08,8.681129e+06,1.940,2.109,1.419,1.926
4,2018-07-07,1.796304,1.091235e+09,1.617507e+07,1.989,2.132,1.896,1.945


### Adding daily price change parameters

In [16]:
df["high_low"] = df.high - df.low
df['high_low_pct'] = df.high/df.low 
df['daily_change'] = df.close-df.open
df['daily_change_pct'] = (df.close-df.open)/df.open

### Average true range

Calculated for time periods of 7 and 14 days.

In [17]:
# Calculate the True Range (TR)
df["high-prevclose"] = abs(df.high - df.close.shift(1))
df['low-prevclose'] = abs(df.low - df.close.shift(1))
df['true_range'] = df[['high_low', 'high-prevclose', 'low-prevclose']].max(axis=1)

df.drop(['high-prevclose', 'low-prevclose'], axis=1, inplace=True)

# Calculate the Average True Range (ATR)
df['ATR_7'] = df.true_range.rolling(window=7).mean()
df['ATR_14'] = df.true_range.rolling(window=14).mean()


### Rate of change (ROC)

Calculated for lookback periods of 3, 7 and 30 days

In [18]:
df['ROC_3'] = (df.close / df.close.shift(3) - 1) * 100
df['ROC_7'] = (df.close / df.close.shift(7) - 1) * 100
df['ROC_30'] = (df.close / df.close.shift(30) - 1) * 100


### Moving averages

Here 2 types of moving averages are chosen: simple moving average and exponential moving average. For both MAs there are 3 sizes of the sliding windows: 3, 7 and 30 days 

In [19]:
df['SMA_3'] = df.close.rolling(3).mean()
df['SMA_7'] = df.close.rolling(7).mean()
df['SMA_30'] = df.close.rolling(30).mean()


df['EMA_3'] = df.close.ewm(span=3).mean()
df['EMA_7'] = df.close.ewm(span=7).mean()
df['EMA_30'] = df.close.ewm(span=30).mean()

### Moving Average Convergence Divergence (MACD)

Comprised of the MACD line, the signal line and the histogram

In [20]:
df["MACD"] = df.EMA_7 - df.EMA_30

signal_period = 9
df["MACD_signal"] = df.MACD.ewm(span=signal_period, adjust=False).mean()
df["MACD_hist"] = df.MACD - df.MACD_signal


### Relative strength index (RSI)

Relative strength index is calculated for look back periods of 7 and 14 days.

In [21]:
def rsi(df, look_back):
    dfc = df.copy()
    dfc["priceDiff"] = dfc.close.diff()

    dfc['Gain'] = dfc.priceDiff.where(dfc.priceDiff > 0, 0)
    dfc['Loss'] = -dfc.priceDiff.where(dfc.priceDiff < 0, 0)

    avg_gain = dfc['Gain'].rolling(window=look_back).mean()
    avg_loss = dfc['Loss'].rolling(window=look_back).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

In [22]:
df['RSI_7'] = rsi(df, 7)
df['RSI_14'] = rsi(df, 14)

### Rolling standard deviation

Calculated for rolling windows of 3, 7 and 30 days.

In [23]:
df['STD_3'] = df.close.rolling(3).std()
df['STD_7'] = df.close.rolling(7).std()
df['STD_30'] = df.close.rolling(30).std()

In [25]:
df.columns

Index(['date', 'current_price', 'market_cap', 'total_volume', 'close', 'high',
       'low', 'open', 'high_low', 'high_low_pct', 'daily_change',
       'daily_change_pct', 'true_range', 'ATR_7', 'ATR_14', 'ROC_3', 'ROC_7',
       'ROC_30', 'SMA_3', 'SMA_7', 'SMA_30', 'EMA_3', 'EMA_7', 'EMA_30',
       'MACD', 'MACD_signal', 'MACD_hist', 'RSI_7', 'RSI_14', 'STD_3', 'STD_7',
       'STD_30'],
      dtype='object')

### Save the results

In [27]:
df.to_csv("../../Data/Tezos/Final/MarketAndPriceWithTI.csv", index=False)
df.to_csv("../../Data/Tezos/DataFullTimePeriod/MarketAndPriceWithTI.csv", index=False)
